In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms as TA_T
from torchvision.transforms import v2 as T
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, auc
)
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings
from urllib.request import urlretrieve

# --- Previous code (Imports, Device, Seed, Checkpoint Download, Augmentations) remains the same ---
warnings.simplefilter(action='ignore', category=FutureWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [2]:
class TimeShift(nn.Module):
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            shift_amount = int(torch.rand(1).item() * self.max_shift_percent * time_steps)
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            if shift_amount != 0:
                 return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(nn.Module):
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            noise = torch.randn_like(spec) * self.noise_level * torch.std(spec)
            return spec + noise
        return spec

train_transform = T.Compose([
    TA_T.TimeMasking(time_mask_param=80),
    TA_T.FrequencyMasking(freq_mask_param=25),
    TimeShift(p=0.5, max_shift_percent=0.4),
    SpecNoise(p=0.3, noise_level=0.03),
    T.Resize((128, 626), antialias=True),
    T.Normalize(mean=[0.0], std=[1.0])
])

val_transform = T.Compose([
    T.Resize((128, 626), antialias=True),
    T.Normalize(mean=[0.0], std=[1.0])
])
print("Đã định nghĩa pipeline Augmentation cho PANNs CNN10.")

Đã định nghĩa pipeline Augmentation cho PANNs CNN10.


In [3]:
CHECKPOINT_URL = "https://zenodo.org/record/3987831/files/Cnn10_mAP=0.380.pth?download=1"
CHECKPOINT_PATH = "Cnn10_mAP=0.380.pth"
if not os.path.exists(CHECKPOINT_PATH):
    print("Downloading pretrained CNN10 checkpoint...")
    urlretrieve(CHECKPOINT_URL, CHECKPOINT_PATH)

In [4]:
class ConvBlock_PANNs(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # Lớp Convolutional thứ nhất trong block
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        # Lớp Convolutional thứ hai trong block
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x, pool_size=(2, 2), pool_type='avg'):
        # Áp dụng lớp conv1 -> bn1 -> relu
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # Áp dụng lớp conv2 -> bn2 -> relu
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        # Áp dụng pooling
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max': # Một số phiên bản PANNs dùng cách này
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Pool type không hợp lệ!')
        return x

# --- Model Cnn10 theo chuẩn PANNs ---
class Cnn10_PANNs(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        # BatchNorm ban đầu - PANNs gốc thường áp dụng cho chiều Freq (Mel bins)
        # Input của bạn là (B, 1, Freq, Time). BN2d(1) sẽ áp dụng cho kênh.
        self.bn0 = nn.BatchNorm2d(1)

        # 4 Khối Convolutional chính
        # Pooling (2,2) sau mỗi khối sẽ giảm Freq và Time
        self.conv_block1 = ConvBlock_PANNs(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock_PANNs(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock_PANNs(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock_PANNs(in_channels=256, out_channels=512)

        # Lớp Fully Connected đầu tiên (sau global pooling)
        # Input size phụ thuộc vào cách global pooling, PANNs thường là 512 hoặc 1024
        # Ở đây, sau mean+max pooling across time và mean across freq -> 512 features
        self.fc1 = nn.Linear(512, 512, bias=True)
        # Lớp Fully Connected cuối cùng (phân loại)
        self.fc_final = nn.Linear(512, num_classes, bias=True)

        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.5) # Có thể thêm dropout nếu cần

        if pretrained:
            self.load_pretrained()

    def load_pretrained(self):
        try:
            checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
            # state_dict thường nằm trong key 'model' hoặc 'state_dict'
            if 'model' in checkpoint:
                state_dict = checkpoint['model']
            elif 'state_dict' in checkpoint:
                 state_dict = checkpoint['state_dict']
            else:
                 state_dict = checkpoint # Nếu file chỉ chứa state_dict

            model_dict = self.state_dict()
            loaded_count = 0
            skipped_mismatch = 0
            skipped_other = 0

            # Tạo dict mới để chứa các trọng số khớp
            pretrained_dict = {}
            for k_ckpt, v_ckpt in state_dict.items():
                # Bỏ qua các lớp không dùng đến (tiền xử lý, lớp phân loại gốc AudioSet)
                if k_ckpt.startswith('spectrogram_extractor.') or \
                   k_ckpt.startswith('logmel_extractor.') or \
                   k_ckpt.startswith('spec_augment.') or \
                   k_ckpt.startswith('fc_audioset.'): # Tên lớp cuối của PANNs gốc
                    skipped_other += 1
                    continue

                # Map tên từ checkpoint sang model hiện tại
                k_model = k_ckpt # Giả sử tên khớp trực tiếp cho bn0, conv_blockX, fc1

                if k_model in model_dict:
                    if model_dict[k_model].shape == v_ckpt.shape:
                        pretrained_dict[k_model] = v_ckpt
                        loaded_count += 1
                    else:
                        # print(f"Skipping {k_ckpt}: Shape mismatch ({model_dict[k_model].shape} vs {v_ckpt.shape})")
                        skipped_mismatch += 1
                else:
                    # print(f"Skipping {k_ckpt}: Key not found in model")
                    skipped_other += 1

            # Cập nhật state_dict của model với các trọng số đã load
            model_dict.update(pretrained_dict)
            # Load state_dict với strict=False để bỏ qua các lớp không khớp (như fc_final)
            missing_keys, unexpected_keys = self.load_state_dict(model_dict, strict=False)

            print(f"Pretrained weights loaded: {loaded_count} tensors loaded.")
            if skipped_mismatch > 0: print(f"Skipped {skipped_mismatch} tensors due to shape mismatch.")
            if skipped_other > 0: print(f"Skipped {skipped_other} tensors (unused layers or key not found).")

            # In ra các key bị thiếu (ngoại trừ lớp cuối cùng là bình thường)
            final_missing = [k for k in missing_keys if not k.startswith('fc_final.')]
            if final_missing:
                print("\nWarning: Missing keys beyond the final classifier:")
                print(final_missing[:10], "..." if len(final_missing) > 10 else "")
            # Unexpected keys lý tưởng là rỗng vì chúng ta đã lọc pretrained_dict
            if unexpected_keys:
                 print("\nWarning: Unexpected keys found (potentially due to model_dict update):")
                 print(unexpected_keys[:10], "...")

        except Exception as e:
            print(f"Error loading pretrained weights: {e}. Training from scratch.")

    def forward(self, x):
        # Input: (B, C=1, F=128, T=626)
        if x.dim() == 3: x = x.unsqueeze(1)

        x = self.bn0(x) # Áp dụng BN ban đầu

        # Qua 4 khối ConvBlock với pooling
        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        # Output dự kiến: (B, 512, F'=8, T'=39)

        # Global Mean + Max Pooling dọc theo chiều Time (dim=3)
        x_mean = torch.mean(x, dim=3) # (B, 512, 8)
        x_max, _ = torch.max(x, dim=3) # (B, 512, 8)
        x = x_mean + x_max # (B, 512, 8)

        # Pooling hoặc Flatten chiều Freq (dim=2) để đưa về (B, 512) cho fc1
        # Ở đây dùng mean pooling dọc theo chiều Freq
        x = torch.mean(x, dim=2) # Output: (B, 512)

        # Lớp Fully Connected
        x = F.relu(self.fc1(x))
        # x = self.dropout(x) # Có thể thêm dropout ở đây
        x = self.fc_final(x)

        return x

print("Đã định nghĩa mô hình Cnn10_PANNs khớp hơn với checkpoint.")

Đã định nghĩa mô hình Cnn10_PANNs khớp hơn với checkpoint.


In [5]:
class SpectrogramDataset(Dataset):
    def __init__(self, metadata_df, melspec_dir, transform=None, target_tdim=626):
        super().__init__()
        self.metadata = metadata_df
        self.melspec_dir = melspec_dir
        self.transform = transform
        self.target_tdim = target_tdim # Target time dimension
        # Ensure label column exists before accessing unique()
        if 'label' in self.metadata.columns:
            self.class_names = sorted(self.metadata['label'].unique())
            self.class_to_id = {name: i for i, name in enumerate(self.class_names)}
        else:
            # Handle error: label column missing
            raise ValueError("Metadata DataFrame must contain a 'label' column.")
        self.error_log = []

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        if idx >= len(self.metadata):
             self.error_log.append(f"Index {idx} out of bounds for metadata length {len(self.metadata)}")
             return None
        # Ensure required columns exist before accessing
        if 'file_path' not in self.metadata.columns or 'label' not in self.metadata.columns:
             self.error_log.append(f"Missing 'file_path' or 'label' column at index {idx}")
             return None

        row = self.metadata.iloc[idx]
        relative_path = row['file_path']
        label_name = row['label']

        # Check if label exists in class_to_id map
        if label_name not in self.class_to_id:
             self.error_log.append(f"Label '{label_name}' at index {idx} not found in class map.")
             return None
        label = self.class_to_id[label_name]

        file_path = os.path.join(self.melspec_dir, relative_path)

        if not os.path.exists(file_path):
             self.error_log.append(f"File not found: {file_path}")
             return None

        try:
            loaded_object = torch.load(file_path, map_location=torch.device('cpu'))

            if isinstance(loaded_object, (tuple, list)):
                if len(loaded_object) > 0 and isinstance(loaded_object[0], torch.Tensor):
                    data = loaded_object[0]
                else:
                    raise TypeError(f"Loaded tuple/list does not contain a tensor at index 0.")
            elif isinstance(loaded_object, torch.Tensor):
                data = loaded_object
            else:
                 raise TypeError(f"Unexpected data type loaded: {type(loaded_object)}")

            data = data.to(torch.float32)

            if data.dim() == 1:
                 self.error_log.append(f"Warning: Loaded data is 1D for {file_path}. Shape: {data.shape}. Skipping.")
                 return None
            elif data.dim() > 2:
                 if data.shape[0] == 1:
                     data = data.squeeze(0)
                 elif data.dim() == 3 and data.shape[-1] == 1:
                     data = data.squeeze(-1)
                 else:
                    self.error_log.append(f"Warning: Loaded data has >2 dims for {file_path}. Shape: {data.shape}. Skipping.")
                    return None

            if data.dim() != 2:
                 self.error_log.append(f"Warning: Data is not 2D after processing for {file_path}. Shape: {data.shape}. Skipping.")
                 return None

            current_tdim = data.shape[-1]
            if current_tdim > self.target_tdim:
                start = (current_tdim - self.target_tdim) // 2
                data = data[..., start:start + self.target_tdim]
            elif current_tdim < self.target_tdim:
                data = F.pad(data, (0, self.target_tdim - current_tdim))

            if self.transform:
                data = data.unsqueeze(0) # Add channel dim -> (1, Freq, Time)
                data = self.transform(data) # Output shape (1, 128, 626)

            return data, label
        except Exception as e:
            self.error_log.append(f"Error loading/processing {file_path}: {str(e)}")
            return None

    def get_error_log(self):
        return self.error_log

In [6]:
def collate_fn_remove_none(batch):
    batch = [item for item in batch if item is not None and item[0] is not None] # Ensure tensor exists
    if not batch:
        return torch.tensor([], device=device, dtype=torch.float32), torch.tensor([], device=device, dtype=torch.long)
    samples, labels = zip(*batch)
    try:
        samples_stacked = torch.stack(samples)
    except RuntimeError as e:
        print(f"Error during torch.stack in collate_fn: {e}")
        for i, s in enumerate(samples): print(f"Sample {i} shape: {s.shape}")
        return torch.tensor([], device=device, dtype=torch.float32), torch.tensor([], device=device, dtype=torch.long)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    return samples_stacked, labels_tensor

In [7]:
def plot_roc_curve(val_labels, val_probs, class_names, save_path):
    n_classes = len(class_names)
    if n_classes <= 1:
        print("ROC curve plotting skipped: requires at least 2 classes.")
        return
    val_labels_np = np.array(val_labels)
    val_probs_np = np.array(val_probs)
    if val_probs_np.shape[1] != n_classes:
         print(f"Warning: Prob columns ({val_probs_np.shape[1]}) != classes ({n_classes}). ROC incorrect.")
    fpr, tpr, roc_auc = dict(), dict(), dict()
    try:
        val_labels_one_hot = F.one_hot(torch.tensor(val_labels_np, dtype=torch.long), num_classes=n_classes).numpy()
    except Exception as e:
        print(f"Error creating one-hot labels: {e}. Skipping ROC plot.")
        return
    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        try:
            fpr[i], tpr[i], _ = roc_curve(val_labels_one_hot[:, i], val_probs_np[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc[i]:.4f})')
        except ValueError as e:
            print(f"Could not compute ROC curve for class {class_names[i]}: {e}")
            roc_auc[i] = float('nan')
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')
    plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve (One-vs-Rest)'); plt.legend(loc="lower right"); plt.grid(True)
    try: plt.savefig(save_path)
    except Exception as e: print(f"Error saving ROC plot: {e}")
    plt.close()

# --- Training and Evaluation Function (train_and_evaluate) remains the same ---
def train_and_evaluate( model, train_loader, val_loader, epochs, results_dir, run_base_name, class_names ):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.2, patience=5, verbose=True)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    best_f1_weighted = 0.0
    best_metrics_dict = {}
    os.makedirs(results_dir, exist_ok=True)
    metrics_csv_path = os.path.join(results_dir, f"metrics_history_{run_base_name}.csv")
    best_checkpoint_path = os.path.join(results_dir, f"Best_{run_base_name}.pth")
    report_path = os.path.join(results_dir, f"Best_{run_base_name}_report.txt")
    cm_path = os.path.join(results_dir, f"Best_{run_base_name}_confusion_matrix.csv")
    auc_plot_path = os.path.join(results_dir, f"Best_{run_base_name}_AUC_ROC.png")
    csv_header = ['Epoch', 'LR', 'Train_Loss', 'Accuracy', 'Precision_micro', 'Precision_macro', 'Precision_w', 'Recall_micro', 'Recall_macro', 'Recall_w', 'F1_micro', 'F1_macro', 'F1_w', 'AUC_macro_ovr']
    if not os.path.exists(metrics_csv_path):
        pd.DataFrame(columns=csv_header).to_csv(metrics_csv_path, index=False)

    print(f"\nStarting training for {run_base_name}...")
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0; num_train_batches = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for inputs, labels in loop:
            if inputs.shape[0] == 0: continue
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item(); num_train_batches += 1
            loop.set_postfix(loss=loss.item())
        avg_train_loss = train_loss / num_train_batches if num_train_batches > 0 else 0
        current_lr = optimizer.param_groups[0]['lr']
        # print(f"Epoch {epoch+1}: Avg Train Loss = {avg_train_loss:.4f}, LR = {current_lr:.1e}") # Less verbose

        model.eval()
        val_preds, val_labels, val_probs = [], [], []
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        with torch.no_grad():
            for inputs, labels in val_loop:
                if inputs.shape[0] == 0: continue
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    outputs = model(inputs)
                probs = F.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_probs.extend(probs.cpu().numpy())

        if not val_labels:
            print(f"Epoch {epoch+1}: No validation samples processed. Skipping metrics.")
            continue
        val_labels_np = np.array(val_labels); val_preds_np = np.array(val_preds); val_probs_np = np.array(val_probs)
        metrics_data = {'Epoch': epoch + 1, 'LR': current_lr, 'Train_Loss': avg_train_loss}
        try:
             metrics_data['Accuracy'] = accuracy_score(val_labels_np, val_preds_np)
             metrics_data['Precision_micro'] = precision_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['Precision_macro'] = precision_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['Precision_w'] = precision_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             metrics_data['Recall_micro'] = recall_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['Recall_macro'] = recall_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['Recall_w'] = recall_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             metrics_data['F1_micro'] = f1_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['F1_macro'] = f1_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['F1_w'] = f1_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             if len(np.unique(val_labels_np)) > 1 and val_probs_np.shape[1] == len(class_names):
                try: metrics_data['AUC_macro_ovr'] = roc_auc_score(val_labels_np, val_probs_np, multi_class='ovr', average='macro')
                except ValueError as auc_error: metrics_data['AUC_macro_ovr'] = 0.0; print(f"AUC Calc Warn: {auc_error}")
             else: metrics_data['AUC_macro_ovr'] = 0.0
        except Exception as metric_error: print(f"Metric Calc Err: {metric_error}"); continue
        try:
             ordered_metrics = {h: metrics_data.get(h, None) for h in csv_header}
             pd.DataFrame([ordered_metrics]).to_csv(metrics_csv_path, mode='a', header=False, index=False)
        except Exception as csv_error: print(f"CSV Write Err: {csv_error}")

        current_f1_weighted = metrics_data.get('F1_w', 0.0)
        print(f"Epoch {epoch+1}: F1-W = {current_f1_weighted:.4f}, F1-M = {metrics_data.get('F1_macro', 0.0):.4f}, Acc = {metrics_data.get('Accuracy', 0.0):.4f}, AUC = {metrics_data.get('AUC_macro_ovr', 0.0):.4f}")

        if current_f1_weighted > best_f1_weighted:
            best_f1_weighted = current_f1_weighted
            best_metrics_dict = metrics_data.copy()
            print(f"  -> New best F1-weighted: {best_f1_weighted:.4f}. Saving artifacts.")
            try: torch.save(model.state_dict(), best_checkpoint_path)
            except Exception as e: print(f"Save Err: {e}")
            try:
                report = classification_report(val_labels_np, val_preds_np, target_names=class_names, digits=4, zero_division=0)
                with open(report_path, 'w', encoding='utf-8') as f:
                    f.write(f"--- Best Metrics (Epoch {best_metrics_dict.get('Epoch', 'N/A')}) ---\n")
                    for k, v in best_metrics_dict.items():
                        if k not in ['Epoch', 'LR', 'Train_Loss']: f.write(f"{k}: {v:.4f}\n")
                    f.write("\n--- Classification Report ---\n"); f.write(report)
            except Exception as e: print(f"Report Save Err: {e}")
            try:
                cm_df = pd.DataFrame(confusion_matrix(val_labels_np, val_preds_np), index=class_names, columns=class_names)
                cm_df.to_csv(cm_path)
            except Exception as e: print(f"CM Save Err: {e}")
            if metrics_data.get('AUC_macro_ovr', 0.0) > 0.0:
                try: plot_roc_curve(val_labels_np, val_probs_np, class_names, auc_plot_path)
                except Exception as e: print(f"ROC Plot Err: {e}")
        scheduler.step(current_f1_weighted)

    print(f"\nFinished Training for {run_base_name}. Best F1-weighted: {best_f1_weighted:.4f} at Epoch {best_metrics_dict.get('Epoch', 'N/A')}")
    train_errors = train_loader.dataset.get_error_log()
    if train_errors:
         print(f"\n--- Train Dataset Errors ({len(train_errors)} files) ---"); [print(err) for err in train_errors[:10]]; print("..." if len(train_errors)>10 else "")
    val_errors = val_loader.dataset.get_error_log()
    if val_errors:
         print(f"\n--- Val Dataset Errors ({len(val_errors)} files) ---"); [print(err) for err in val_errors[:10]]; print("..." if len(val_errors)>10 else "")
    return best_metrics_dict

In [8]:
MELSPEC_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\Finetune_MelSpec' 
METADATA_PATH = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\metadata_Finetune_MelSpec.csv'
BASE_OUTPUT_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare'
print(f"Tất cả kết quả sẽ được lưu vào các thư mục con trong: {BASE_OUTPUT_DIR}")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
DATA_PERCENTAGES = [1.0]
results = {} 

Tất cả kết quả sẽ được lưu vào các thư mục con trong: C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare


In [9]:
print("Preparing dataset...")
try:
    full_metadata = pd.read_csv(METADATA_PATH)
except FileNotFoundError: print(f"Error: Metadata file not found at {METADATA_PATH}"); exit()
if 'file_path' not in full_metadata.columns or 'label' not in full_metadata.columns:
    print("Error: Metadata CSV must contain 'file_path' and 'label' columns."); exit()
initial_len = len(full_metadata)
full_metadata.dropna(subset=['file_path', 'label'], inplace=True)
if len(full_metadata) < initial_len: print(f"Warning: Dropped {initial_len - len(full_metadata)} rows with missing values.")

num_classes = len(full_metadata['label'].unique())
if num_classes == 0: print("Error: No classes found."); exit()
print(f"Số classes: {num_classes}")

indices = np.arange(len(full_metadata)); labels = full_metadata['label'].values
try:
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=SEED, stratify=labels)
    print(f"Split: {len(train_indices)} train, {len(test_indices)} test.")
except ValueError as e:
     print(f"Stratify Error: {e}. Using non-stratified split."); train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=SEED)
     print(f"Split: {len(train_indices)} train, {len(test_indices)} test.")

test_metadata = full_metadata.iloc[test_indices].copy()
test_dataset = SpectrogramDataset(test_metadata, MELSPEC_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=collate_fn_remove_none, pin_memory=torch.cuda.is_available())
try:
    class_names_list = test_dataset.class_names
    print(f"Class names: {class_names_list}")
except AttributeError:
    # Handle case where test_dataset might not have class_names if initialization failed
    print("Error: Could not retrieve class names from test_dataset.")
    exit()


model_configs = {
    "PANNs-CNN10": {
        "model_class": Cnn10_PANNs, # <--- Sử dụng lớp mới
        "args": {"num_classes": num_classes, "pretrained": True}
    }
}

Preparing dataset...
Số classes: 8
Split: 11240 train, 2810 test.
Class names: ['Gryllus_bimaculatus', 'Gryllus_campestris', 'Halyomorpha', 'Melolontha_melolontha', 'Myzus', 'Nezara', 'Trialeurodes', 'Tuta']


In [10]:
for model_name, config in model_configs.items():
    print(f"\n{'='*20} Evaluating: {model_name} {'='*20}")
    model_results = []
    model_folder_name = model_name.replace(' ', '_').replace('-', '_')
    model_output_dir = os.path.join(BASE_OUTPUT_DIR, model_folder_name)
    os.makedirs(model_output_dir, exist_ok=True)
    print(f"Lưu kết quả của model này vào: {model_output_dir}")

    for percent in DATA_PERCENTAGES:
        print(f"\n--- Training with {percent*100:.1f}% of labeled data ---")
        if percent < 1.0:
             train_labels_for_stratify = full_metadata['label'].iloc[train_indices].values
             min_samples_per_class = pd.Series(train_labels_for_stratify).value_counts().min()
             required_samples = 2
             if min_samples_per_class >= required_samples:
                subset_train_indices, _ = train_test_split(train_indices, train_size=percent, random_state=SEED, stratify=train_labels_for_stratify)
             else:
                 print(f"Warning: Cannot stratify subset. Using non-stratified subset.")
                 subset_train_indices, _ = train_test_split(train_indices, train_size=percent, random_state=SEED)
        else:
            subset_train_indices = train_indices

        train_metadata_subset = full_metadata.iloc[subset_train_indices].copy()
        train_dataset = SpectrogramDataset(train_metadata_subset, MELSPEC_DIR, transform=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none, pin_memory=torch.cuda.is_available())
        print(f"Train dataset size: {len(train_dataset)}")
        print(f"Test dataset size: {len(test_dataset)}")

        percent_str = f"data_{percent*100:.0f}percent"
        run_output_dir = os.path.join(model_output_dir, percent_str)
        os.makedirs(run_output_dir, exist_ok=True)
        run_base_name = f"{model_folder_name}_{percent_str}"

        model = config["model_class"](**config["args"]).to(device)

        best_metrics = train_and_evaluate( model, train_loader, test_loader, epochs=100, results_dir=run_output_dir, run_base_name=run_base_name, class_names=class_names_list )

        model_results.append({'percent': percent, 'metrics': best_metrics})
        if best_metrics: print(f"==> Results with {percent*100:.1f}% data: Best F1-Weighted = {best_metrics.get('F1_w', 0):.4f}")
        else: print(f"==> No valid metrics recorded for {percent*100:.1f}% data run.")
    results[model_name] = model_results

print("\n--- FINAL RESULTS ---")
for model_name, result_list in results.items():
    print(f"\nModel: {model_name}")
    if not result_list: print("  No results recorded."); continue
    for res in result_list:
        percent_str = f"{res['percent']*100:.1f}% data"
        if res['metrics']:
            f1_w = res['metrics'].get('F1_w', float('nan')); f1_m = res['metrics'].get('F1_macro', float('nan'))
            auc = res['metrics'].get('AUC_macro_ovr', float('nan')); acc = res['metrics'].get('Accuracy', float('nan'))
            epoch = res['metrics'].get('Epoch', 'N/A')
            print(f"  {percent_str} (Epoch {epoch}): F1-W={f1_w:.4f} | F1-M={f1_m:.4f} | AUC={auc:.4f} | Acc={acc:.4f}")
        else: print(f"  {percent_str}: No valid metrics recorded.")


==================== Evaluating: PANNs-CNN10 ====================
Lưu kết quả của model này vào: C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare\PANNs_CNN10

--- Training with 100.0% of labeled data ---
Train dataset size: 11240
Test dataset size: 2810
Pretrained weights loaded: 51 tensors loaded.
Skipped 4 tensors due to shape mismatch.
Skipped 5 tensors (unused layers or key not found).

Starting training for PANNs_CNN10_data_100percent...


C:\Users\PC\anaconda3\envs\train_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100 [Train]:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 1/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 1: F1-W = 0.5568, F1-M = 0.4924, Acc = 0.5904, AUC = 0.0000
  -> New best F1-weighted: 0.5568. Saving artifacts.


...

Epoch 99/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 99: F1-W = 0.8674, F1-M = 0.8562, Acc = 0.8673, AUC = 0.0000


Epoch 100/100 [Train]:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 100/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 100: F1-W = 0.8629, F1-M = 0.8523, Acc = 0.8626, AUC = 0.0000

Finished Training for PANNs_CNN10_data_100percent. Best F1-weighted: 0.8692 at Epoch 57
==> Results with 100.0% data: Best F1-Weighted = 0.8692

--- FINAL RESULTS ---

Model: PANNs-CNN10
  100.0% data (Epoch 57): F1-W=0.8692 | F1-M=0.8588 | AUC=0.0000 | Acc=0.8690
